# NOTEBOOK FOURRE-TOUT pour scrips PlaMADE

In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append(r'C:\Users\martin.schoreisz\git\otv\otv\Transfert_Donnees')
sys.path.append(r'C:\Users\martin.schoreisz\git\Outils\Outils\Martin_Perso')
import os, re
import pandas as pd
import geopandas as gp
import numpy as np
import zipfile
import Connexion_Transfert as ct
import Outils as O
from geoalchemy2 import Geometry,WKTElement
from shapely.geometry import Point, LineString
from shapely.geometry.point import Point
from shapely.geometry.multipoint import MultiPoint

c:\users\martin.schoreisz\appdata\local\programs\python\python38\lib\site-packages\geopandas\_compat.py:84: UserWarning: The Shapely GEOS version (3.4.3-CAPI-1.8.3 r4285) is incompatible with the GEOS version PyGEOS was compiled with (3.8.1-CAPI-1.13.3). Conversions between both will be slow.
  warnings.warn(


In [9]:
from shapely import speedups
speedups.disable()

In [ ]:
dossierSrc=r'C:\Users\martin.schoreisz\Box\Donnees_source\Donnees_geostandardisees'
#dézipper tous les dossier de Gérard
for root,dirs, files in os.walk(dossierSrc) : 
    for f in files : 
        if f.endswith('.zip') : 
            cheminFichier=os.path.join(root, f)
            print(cheminFichier[:-4])
            try:
                with zipfile.ZipFile(cheminFichier) as z:
                    z.extractall(cheminFichier[:-4])
                    print(f"fichier extrait {cheminFichier[:-4]}")
            except:
                print(f"pb extraction sur dossier {cheminFichier[:-4]}")

## 1. creer une bdd des fichiers geostandardises

In [ ]:
#pour traiter tous les fichiers d'un dossier général.suppose que la structure des tables a déjà été crée (par exemple avec l'import des des fichiers pouis truncate)
dossierSrc=r'D:\Boulot\PlaMADE\Ile-de-france\75-Paris\75'
coupleFichierTable=(('N_ROUTIER_ALLURE','allure_national'),('N_ROUTIER_REVETEMENT','rvt_national'),('N_ROUTIER_ROUTE','route_national'),
                    ('N_ROUTIER_TRAFIC','trafic_national'),('N_ROUTIER_VITESSE','vts_national'))
#dossierSrc=r'C:\Users\martin.schoreisz\Box\Donnees_source\Donnees_geostandardisees\Auvergne-Rhone-Alpes\Donnees_geostandardisees\Route01_v2_dec2020'
listEreur=[]
with ct.ConnexionBdd('local_PlaMADE', 'maison') as c :
    for root,dirs, files in os.walk(dossierSrc) : 
        for f in files :
            if f.endswith(('.shp', '.dbf')) : 
                print(os.path.join(root,f))
                if f.endswith('.shp') and 'N_ROUTIER_TRONCON' in f :
                    try :
                        ct.ogr2ogr_shp2pg(c.connstringOgr,os.path.join(root,f),
                                              schema='geostandardise_src', table='troncon_national',
                                              SRID=None,geotype='MULTILINESTRINGZ', dims=3, creationMode='-append -update',encodageClient='UTF-8', version_simple=True)
                    except Exception as e: 
                        listEreur.append({'fichier': f, erreur : e})
                else : 
                    for fich,t in coupleFichierTable :
                        try : 
                            if fich in f.upper() and f.endswith('.dbf') : 
                                ct.ogr2ogr_shp2pg(c.connstringOgr,os.path.join(root,f),
                                                  schema='geostandardise_src', table=t, SRID=None,geotype=None, dims=None, creationMode='-append -update',encodageClient='UTF-8', requeteSql='', version_simple=True)
                            elif fich in f.upper() and f.endswith('.csv') :
                                df = pd.read_csv(os.path.join(root,f), 
                                                 keep_default_na=False)
                                df.columns=[c.lower() for c in df.columns]
                                df.drop(colonnesEnTrop,axis=1).to_sql(t,c.sqlAlchemyConn,'geostandardise_src', if_exists='append', index=False )
                        except Exception as e: 
                            listEreur.append({'fichier': f, 'erreur' : e})

In [ ]:
#si les donnees trafic, vts, allure , rvt sont en csv, le plus simple c'est pandas : 
dossierSrc=r'C:\Users\martin.schoreisz\Box\Donnees_source\Donnees_geostandardisees\Bretagne'
coupleFichierTable=(('N_ROUTIER_ALLURE','allure_national'),('N_ROUTIER_REVETEMENT','rvt_national'),('N_ROUTIER_ROUTE','route_national'),
                    ('N_ROUTIER_TRAFIC','trafic_national'),('N_ROUTIER_VITESSE','vts_national'))
with ct.ConnexionBdd('local_PlaMADE', 'maison') as c :
    for root,dirs, files in os.walk(dossierSrc) : 
            for f in files :
                if f.endswith('.csv') :
                    for fich,t in coupleFichierTable :
                        if fich in f.upper() : 
                            try : 
                                print(os.path.join(root,f))
                                df = pd.read_csv(os.path.join(root,f), 
                                                keep_default_na=False)
                                df.columns=[c.lower() for c in df.columns]
                                dfref=pd.read_sql(f'select * from geostandardise_src.{t} limit 1',c.sqlAlchemyConn)
                                colonnesEnTrop=[c.lower() for c in df.columns if c not in dfref.columns]
                                df.drop(colonnesEnTrop,axis=1).to_sql(t,c.sqlAlchemyConn,'geostandardise_src', if_exists='append', index=False )
                            except Exception as e :
                                print(f'Erreur sur : {os.path.join(root,f)} : {e} ')

In [ ]:
#pour test sur un dept
with ct.ConnexionBdd('local_PlaMADE', 'maison') as c :
    ct.ogr2ogr_shp2pg(c.connstringOgr,r'C:\Users\martin.schoreisz\Box\Donnees_source\Donnees_geostandardisees\Bretagne\35\N_ROUTIER_TRONCON_L_035.shp',
                                              schema='geostandardise_src', table='troncon_national',
                                              SRID='2154',geotype='MULTILINESTRING', dims=3, creationMode='-append -update',encodageClient='UTF-8')

In [ ]:
#sur un fichier
with ct.ConnexionBdd('local_PlaMADE', 'maison') as c :
    ct.ogr2ogr_shp2pg(c.connstringOgr,r'C:\Users\martin.schoreisz\Box\Donnees_source\Donnees_geostandardisees\Auvergne_Rhone_Alpes\Donnees_geostandardisees\Route01_v2_dec2020\N_ROUTIER_ALLURE_001.dbf',
                                                  schema='geostandardise_src', table='allure_national', SRID=None,geotype=None, dims=None, creationMode='-append -update',encodageClient='UTF-8', requeteSql='', version_simple=True)


## 2. Générer un fichier unique trafic RD
> L'idée est de se baser sur le fichier de résumé des fichiers gestionnaires et de leurs attributs pour produire un fichier concaténé, avec des attributs uniques relatifs au :
- tmja
- pcpl
- nom de la voie
- annee du trafic
- fichier source (sur Box internet)
- nom de la source

> ### 2.1 Principe mise en forme des fichiers 
>> Pour faire ça on va aller chercher le [fichier de synthses des données RD] (https://cerema.app.box.com/folder/132749694470/Synthese_type_fichier.ods).
dans l'onglet trafic, on va itérer sur chaque ligne (donc récupération du tuple de valeur) : 
>> On limite l'analyse aux données présentant un colonne valide='oui' & type='sig' & type_geom='ligne'  
1. on récupere la valeur de la colonne "nom_fichier_trafic" :
    1. si l'extenstion est présente, on va lire le fichier (attention, via box Drive, donc paramètre de raw string en entrée)
    1. l'attribut relatif au tmja est toujours présent, celui au pc_pl parfois, comme pour les routes ou l'année, mais comme la fonction rename s'en fout, on fait le rename des 4 colonnes
    1. si des formules sont à appliquer, on les applique (liées au pcpl, et nom de route notamment
    1. gérer le format des dates pour l'année de mesure
    1. ajouter les attributs sur le fichiers sources, le nom de la source, le type de source
    1. enregistrer le fichier mise en forme dans le dossier qui va bien (paramètre entrée)

In [412]:
# ouvrir le fichiers de syntheses des données
dossierDonneesRd=r'C:\Users\martin.schoreisz\Box\Projet PLaMADE\PLAMADE\Reprise PlaMADE-Projet Sword\Données\3-RD'
fichierSynthese=r'C:\Users\martin.schoreisz\Box\Projet PLaMADE\PLAMADE\Reprise PlaMADE-Projet Sword\Données\3-RD\Synthese_type_fichier.ods'
dfFichierSynthese=pd.read_excel(fichierSynthese, sheet_name='trafic',engine='odf', nrows=35, dtype={'dept':str})

In [413]:
#creer le generateurde parcours des valeusr en tuple
dfNettoyeesSansReglesGestion=dfFichierSynthese.loc[(dfFichierSynthese['valide']=='oui') & (dfFichierSynthese['type']=='sig') & (dfFichierSynthese['type_geom']=='ligne')
                                 & (dfFichierSynthese['regles_gestion']=='non')].copy()
dfNettoyeesAvecReglesGestion=dfFichierSynthese.loc[(dfFichierSynthese['valide']=='oui') & (dfFichierSynthese['type']=='sig') & (dfFichierSynthese['type_geom']=='ligne')
                                 & (dfFichierSynthese['regles_gestion']!='non')].copy()
dfNettoyees=dfFichierSynthese.loc[(dfFichierSynthese['valide']=='oui') & (dfFichierSynthese['type']=='sig') & (dfFichierSynthese['type_geom']=='ligne')].copy()

In [414]:
#fonction d'ajout des sources de données
def ajouterSourceDonnees(tupleNomAttr, dfFichierRenomme, fihcierSource) : 
    """
    ajouter les nomsource, fichier source et type de source aux donnees mise en formes
    in : 
        tupleNomAttr : tuple des noms d'attributs recherche dans le fchier source
        dfFichierRenomme : gdf : contient la trsucture attributaire prevue, apres calcul
        fihcierSource : String : nom du fichier source
    """
    dfFichierRenomme['nomsource']='CD_'+tupleNomAttr.dept
    dfFichierRenomme['fichie_src']=fihcierSource

In [415]:
#fonction d'ouverture et reprojection en 2154
def ouvrirReprojeter(cheminFichier, tupleNomAttr):
    """
    tupleNomAttr : tuple des noms d'attributs recherche dans le fchier source
    """
    dfFichierSource=gp.read_file(cheminFichier, crs=tupleNomAttr.projection)
    dfFichierSource=dfFichierSource.to_crs('epsg:2154')
    return dfFichierSource
    

In [416]:
#fonction de calcul regroupant les traitements propres a chaque dept ayant des regles de gestion
def calculSpecifique(dfFichierSource, tupleNomAttr, listAttrNonNull) : 
    """
    calculer les valeur de tmja, pcpl, nom route et année dans les cas particuklier ou des regles de gestion sont necessaire
    in : 
        dept : string : departement sur 3 caracteres
        dfFichierSource : gdf : gdf issue des fichiers osurces gestionnaiere
        tupleNomAttr : tuple des noms d'attributs recherche dans le fchier source
        listAttrNonNull : list des attributs present dans le fichier (ayant une valuer non nulle)
    out : 
        dfFichierRenomme : gdf : contient la trsucture attributaire prevue, apres calcul
    """
    if tupleNomAttr.dept=='012' : 
        dfFichierSource['trafic']=dfFichierSource.apply(lambda x : x['TMJATV2015'] if not pd.isnull(x['TMJATV2015']) else x['TMJA_TV'], axis=1)
        dfFichierSource['annee']=dfFichierSource.apply(lambda x : re.match('2[0-9]{3]',x['COMMENTAIR']) if not pd.isnull(x['COMMENTAIR']) else None, axis=1)
        dfFichierRenomme=dfFichierSource.rename(columns={tupleNomAttr.nom_attr_pcpl:'pcpl', tupleNomAttr.nom_attr_route:'nomRoute'})
    elif tupleNomAttr.dept=='025' :
        dfFichierSource['trafic']=dfFichierSource.apply(lambda x : x['TMJA'] if x['%_PL']!='D' else None, axis=1)
        dfFichierSource['pcpl']=dfFichierSource.apply(lambda x : str(x['%_PL']) if x['%_PL']!='D' else None, axis=1)
        dfFichierRenomme=dfFichierSource.rename(columns={tupleNomAttr.nom_attr_route:'nomRoute', t.nom_attr_annee:'annee'})
    elif tupleNomAttr.dept=='027' :
        dfFichierSource['pcpl']=dfFichierSource.F_PL.apply(lambda x : x.replace('%','').replace(',','.') if not pd.isnull(x) else None)
        dfFichierRenomme=dfFichierSource.rename(columns={t.nom_attr_trafic:'trafic', t.nom_attr_annee:'annee',tupleNomAttr.nom_attr_route:'nomRoute'})
    elif tupleNomAttr.dept=='031' :
        dfFichierSource['nomRoute']=dfFichierSource.route.apply(lambda x : O.epurationNomRoute(x))
        dfFichierRenomme=dfFichierSource.rename(columns={t.nom_attr_trafic:'trafic',tupleNomAttr.nom_attr_pcpl:'pcpl', t.nom_attr_annee:'annee'})
    elif tupleNomAttr.dept=='037' :
        dfFichierSource['nomRoute']=dfFichierSource.idroute.apply(lambda x : x.split('_')[1])
        dfFichierRenomme=dfFichierSource.rename(columns={t.nom_attr_trafic:'trafic',tupleNomAttr.nom_attr_pcpl:'pcpl', t.nom_attr_annee:'annee'})
    elif tupleNomAttr.dept=='046': 
        dfFichierSource['trafic']=dfFichierSource['MJA TV (an'].apply(lambda x : x.replace(' ','').split('(')[0] if not pd.isnull(x) else None)
        dfFichierSource['annee']=dfFichierSource['MJA TV (an'].apply(lambda x : x.replace(' ','').split('(')[1][:-1] if not pd.isnull(x) and '(' in x else None)
        dfFichierSource['pcpl']=dfFichierSource['%PL'].apply(lambda x : float(x)*100 if not pd.isnull(x) else None)
        dfFichierSource['nomRoute']=dfFichierSource['RD'].apply(lambda x : x.replace(' ','').replace('RD','D'))
        dfFichierRenomme=dfFichierSource.copy()
    elif tupleNomAttr.dept=='049' :
        dfFichierSource['nomRoute']=dfFichierSource.Route.apply(lambda x : O.epurationNomRoute(x[3:]))
        dfFichierRenomme=dfFichierSource.rename(columns={t.nom_attr_trafic:'trafic',tupleNomAttr.nom_attr_pcpl:'pcpl', t.nom_attr_annee:'annee'})
    elif tupleNomAttr.dept=='052' :
        dfFichierSource['pcpl']=dfFichierSource.apply(lambda x : str(x['MJA_PL']/x['MJA_TV']*100) if not pd.isnull(x['MJA_TV']) else None, axis=1)
        dfFichierSource['nomRoute']=dfFichierSource.Route.apply(lambda x : O.epurationNomRoute(x))
        dfFichierRenomme=dfFichierSource.rename(columns={t.nom_attr_trafic:'trafic', t.nom_attr_annee:'annee'})
    elif tupleNomAttr.dept=='058' :
        dfFichierSource['trafic']=dfFichierSource.apply(lambda x : x['TRAFFIC_VL']+x['TRAFFIC_PL'], axis=1)
        dfFichierSource['pcpl']=dfFichierSource.apply(lambda x : str(x['TRAFFIC_PL']/(x['TRAFFIC_VL']+x['TRAFFIC_PL'])*100) if x['TRAFFIC_VL']+x['TRAFFIC_PL']>0 else None, axis=1)
        dfFichierRenomme=dfFichierSource.rename(columns={tupleNomAttr.nom_attr_route:'nomRoute', t.nom_attr_annee:'annee'})
    elif tupleNomAttr.dept=='059' :
        dfFichierSource['nomRoute']=dfFichierSource.VOIE.apply(lambda x : O.epurationNomRoute(x[1:]))
        dfFichierRenomme=dfFichierSource.rename(columns={t.nom_attr_trafic:'trafic',tupleNomAttr.nom_attr_pcpl:'pcpl', t.nom_attr_annee:'annee'})
    elif tupleNomAttr.dept=='060' :
        dfFichierSource['pcpl']=dfFichierSource.PCT_PL_SEM.apply(lambda x : x.replace('%PL','') if not x=='%PL' else None)
        dfFichierSource['annee']=dfFichierSource.DATE_COMPT.str[-4:]
        dfFichierRenomme=dfFichierSource.rename(columns={t.nom_attr_trafic:'trafic',tupleNomAttr.nom_attr_route:'nomRoute'})
    elif tupleNomAttr.dept=='070':
        for c in [f'TA_TV{i}' for i in range(10,19)]: 
            dfFichierSource.loc[dfFichierSource.apply(lambda x : float(x[c])>0 if not pd.isnull(x[c]) else False, axis=1),'trafic']= dfFichierSource[c]
            dfFichierSource.loc[dfFichierSource.apply(lambda x : float(x[c.replace('TV','PL')])>0 if not pd.isnull(x[c.replace('TV','PL')]) else False, axis=1),'pcpl']=(dfFichierSource.loc[dfFichierSource.apply(lambda x : float(x[c.replace('TV','PL')])>0 if not pd.isnull(x[c.replace('TV','PL')]) else False, axis=1)].apply(lambda x : float(x[c.replace('TV','PL')]), axis=1)/
              dfFichierSource.loc[dfFichierSource.apply(lambda x : float(x[c.replace('TV','PL')])>0 if not pd.isnull(x[c.replace('TV','PL')]) else False, axis=1)].apply(lambda x : float(x[c]), axis=1)*100)                
            dfFichierSource.loc[dfFichierSource.apply(lambda x : float(x[c])>0 if not pd.isnull(x[c]) else False , axis=1),'annee']= f'20{c[-2:]}'
            dfFichierRenomme=dfFichierSource.rename(columns={tupleNomAttr.nom_attr_route:'nomRoute'})
    elif tupleNomAttr.dept=='075' :
        dfFichierSource['trafic']=dfFichierSource.apply(lambda x : (x['MT']*12)+(x['ME']*4)+(x['MN']*8) if not (pd.isnull(x['MT']) and pd.isnull(x['ME']) and pd.isnull(x['MN'])) else None, axis=1)
        dfFichierRenomme=dfFichierSource.rename(columns={tupleNomAttr.nom_attr_pcpl:'pcpl', tupleNomAttr.nom_attr_route:'nomRoute', t.nom_attr_annee:'annee'})
    elif tupleNomAttr.dept in ('080','082') :
        dfFichierSource['nomRoute']=dfFichierSource[tupleNomAttr.nom_attr_route].str[3:]
        dfFichierRenomme=dfFichierSource.rename(columns={t.nom_attr_trafic:'trafic',tupleNomAttr.nom_attr_pcpl:'pcpl', t.nom_attr_annee:'annee'})
    elif tupleNomAttr.dept =='076' : 
        dfFichierSource['nomRoute']=dfFichierSource[tupleNomAttr.nom_attr_route].apply(lambda x : O.epurationNomRoute(x[3:]) if x[:2]!='VC' else x )
        dfFichierRenomme=dfFichierSource.rename(columns={t.nom_attr_trafic:'trafic',tupleNomAttr.nom_attr_pcpl:'pcpl', t.nom_attr_annee:'annee'})
    elif tupleNomAttr.dept=='077' :
        dfFichierSource['pcpl']=dfFichierSource.apply(lambda x : str(x['PLTMJA']/x['TVTMJA']*100) if not pd.isnull(x['TVTMJA']) else None, axis=1)
        dfFichierSource['nomRoute']=dfFichierSource.ROUTE.apply(lambda x : O.epurationNomRoute(x[2:]))
        dfFichierRenomme=dfFichierSource.rename(columns={t.nom_attr_trafic:'trafic', t.nom_attr_annee:'annee'})
    elif tupleNomAttr.dept=='083' :
        dfFichierSource['nomRoute']=dfFichierSource.ROUTE.apply(lambda x : O.epurationNomRoute(x[3:]))
        dfFichierSource['trafic']=dfFichierSource.apply(lambda x : int(x['MJA_2018']) if int(x['MJA_2018'])>0 else int(x['MJA_2017']), axis=1)
        dfFichierSource['pcpl']=dfFichierSource.apply(lambda x : x['PL_2018'] if int(x['MJA_2018'])>0 else str(int(x['PL_2017'])/10), axis=1)
        dfFichierSource['annee']=dfFichierSource.apply(lambda x : '2018' if int(x['MJA_2018'])>0 else '2017', axis=1)                                                                                                  
        dfFichierRenomme=dfFichierSource.copy()
    return dfFichierRenomme[listAttrNonNull]

In [417]:
#Concat des donnees 
dicoGdf={}
for t in dfNettoyees.itertuples() : 
    #ouvrir le fichier
    print(t.dept)
    for root, dirs, files in os.walk(os.path.join(dossierDonneesRd, t.dept)) : 
        for f in files : 
            """if t.dept  not in  ('027') : 
                continue"""
            if f.lower() == t.nom_fichier_trafic.lower() : 
                #ouvrir et modifier le crs si besoin
                dfFichierSource=ouvrirReprojeter(os.path.join(root, f),t)
                dicoNomAttr={t.nom_attr_trafic:'trafic', t.nom_attr_pcpl:'pcpl', t.nom_attr_route:'nomRoute',t.nom_attr_annee:'annee'}
                listAttrNonNull=[v for k,v in dicoNomAttr.items() if not pd.isnull(k)]+['geometry']
                #print(listAttrNonNull)
                #a reprendre apres pour le cas des departements particuliers non traites
                if t.dept in ('012','027', '037', '049', '052', '058', '059', '060', '075', '077','082', '083', '076','080'):
                    dfFichierRenomme=calculSpecifique(dfFichierSource, t, listAttrNonNull)
                else :
                    dfFichierRenomme=dfFichierSource.rename(columns=dicoNomAttr)[listAttrNonNull]
                ajouterSourceDonnees(t, dfFichierRenomme, f)
    dicoGdf[t.dept]=dfFichierRenomme
dfConcat=pd.concat(dicoGdf.values())

012
013
027
035
036
037
044
045
048
049
050
052
058
059
060
066
067
068
071
075
076
077
080
082
083
092


In [418]:
# traitemnets avant export : 
#tagguer les geoms nulles 
dfConcat.loc[dfConcat.geometry.apply(lambda x : not pd.isnull(x)), 'geom_nulle']=False
dfConcat.loc[dfConcat.geometry.apply(lambda x : pd.isnull(x)), 'geom_nulle']=True

#corriger des valeurs louches sur les annees et passer en format date AAAA pour le reste
dfConcat.annee=dfConcat.annee.apply(lambda x : str(x) if not pd.isnull(x) else None)
dfConcat.annee=dfConcat.annee.str[:4]
dfConcat.loc[dfConcat.annee.apply(lambda x : float(x)<1950 if not (pd.isnull(x)) else True), 'annee']=None

#mise en forme valeurs TMJA et pcpl : 
dfConcat.trafic=dfConcat.trafic.apply(lambda x : round(float(x),0) if not pd.isnull(x) else np.nan)
dfConcat.pcpl=dfConcat.pcpl.apply(lambda x : round(float(str(x).replace(',','.')),2) if not pd.isnull(x) else np.nan)

In [419]:
#export
dfConcat.to_file(r'D:\Boulot\AffairesEnCours\plamade\RDs\concat\RD_concatenation_trafic_lineaire.shp')

> ### 2.2 Principe mise en forme des fichiers ponctuels
>> on va garder la structure précédente, mais il va aussi falloir ajouter les 2 jeu de données en tableur

In [405]:
dfNettoyeesSansReglesGestion=dfFichierSynthese.loc[(dfFichierSynthese['valide']=='oui') & (dfFichierSynthese['type']=='sig') & (dfFichierSynthese['type_geom']=='point')
                                 & (dfFichierSynthese['regles_gestion']=='non')].copy()
dfNettoyeesAvecReglesGestion=dfFichierSynthese.loc[(dfFichierSynthese['valide']=='oui') & (dfFichierSynthese['type']=='sig') & (dfFichierSynthese['type_geom']=='point')
                                 & (dfFichierSynthese['regles_gestion']!='non')].copy()
dfNettoyees=dfFichierSynthese.loc[(dfFichierSynthese['valide']=='oui') & (dfFichierSynthese['type']=='sig') & (dfFichierSynthese['type_geom']=='point')].copy()

In [406]:
dicoGdf={}
for t in dfNettoyees.itertuples() : 
    #ouvrir le fichier
    print(t.dept)
    for root, dirs, files in os.walk(os.path.join(dossierDonneesRd, t.dept)) : 
        for f in files : 
            """if t.dept  not in  ('046') : 
                continue"""
            if f.lower() == t.nom_fichier_trafic.lower() : 
                #print(f)
                #ouvrir et modifier le crs si besoin
                dfFichierSource=ouvrirReprojeter(os.path.join(root, f),t)
                dicoNomAttr={t.nom_attr_trafic:'trafic', t.nom_attr_pcpl:'pcpl', t.nom_attr_route:'nomRoute',t.nom_attr_annee:'annee'}
                listAttrNonNull=[v for k,v in dicoNomAttr.items() if not pd.isnull(k)]+['geometry']
                #a reprendre apres pour le cas des departements particuliers non traites
                if t.dept in ('025','031', '070', '046'):
                    dfFichierRenomme=calculSpecifique(dfFichierSource, t, listAttrNonNull)
                else :
                    dfFichierRenomme=dfFichierSource.rename(columns=dicoNomAttr)[listAttrNonNull]
                ajouterSourceDonnees(t, dfFichierRenomme, f)
    dicoGdf[t.dept]=dfFichierRenomme
dfConcatFixe=pd.concat(dicoGdf.values())

002
008
025
031
039
046
070
089


In [408]:
# traitemnets avant export : 
#tagguer les geoms nulles 
dfConcat=dfConcatFixe.copy()

dfConcat.loc[dfConcat.geometry.apply(lambda x : not pd.isnull(x)), 'geom_nulle']=False
dfConcat.loc[dfConcat.geometry.apply(lambda x : pd.isnull(x)), 'geom_nulle']=True

#corriger des valeurs louches sur les annees et passer en format date AAAA pour le reste
dfConcat.annee=dfConcat.annee.apply(lambda x : str(x) if not pd.isnull(x) else None)
dfConcat.annee=dfConcat.annee.str[:4]
dfConcat.loc[dfConcat.annee.apply(lambda x : float(x)<1950 if not (pd.isnull(x)) else True), 'annee']=None
dfConcat.loc[dfConcat.nomsource=='CD_008', 'annee']='2018'

#mise en forme valeurs TMJA et pcpl : 
dfConcat.trafic=dfConcat.trafic.apply(lambda x : round(float(x),0) if not pd.isnull(x) else np.nan)
dfConcat.pcpl=dfConcat.pcpl.apply(lambda x : round(float(str(x).replace(',','.')),2) if not pd.isnull(x) else np.nan)

#toutes les geoms en multi
dfConcat["geometry"] = [MultiPoint([feature]) if isinstance(feature, Point) else feature for feature in dfConcat["geometry"]]

In [409]:
#export
dfConcat.to_file(r'D:\Boulot\AffairesEnCours\plamade\RDs\concat\RD_concatenation_trafic_ponctuel.shp')

In [333]:
for c in [f'TA_TV{i}' for i in range(10,19)]: 
            dfFichierSource.loc[dfFichierSource.apply(lambda x : float(x[c])>0 if not pd.isnull(x[c]) else False, axis=1),'trafic']= dfFichierSource[c]
            dfFichierSource.loc[dfFichierSource.apply(lambda x : float(x[c.replace('TV','PL')])>0 if not pd.isnull(x[c.replace('TV','PL')]) else False, axis=1),'pcpl']=(dfFichierSource.loc[dfFichierSource.apply(lambda x : float(x[c.replace('TV','PL')])>0 if not pd.isnull(x[c.replace('TV','PL')]) else False, axis=1)].apply(lambda x : float(x[c.replace('TV','PL')]), axis=1)/
              dfFichierSource.loc[dfFichierSource.apply(lambda x : float(x[c.replace('TV','PL')])>0 if not pd.isnull(x[c.replace('TV','PL')]) else False, axis=1)].apply(lambda x : float(x[c]), axis=1)*100)                
            dfFichierSource.loc[dfFichierSource.apply(lambda x : float(x[c])>0 if not pd.isnull(x[c]) else False , axis=1),'annee']= f'20{c[-2:]}'

In [324]:
dfFichierSource.loc[pd.isnull(dfFichierSource.annee)]
dfFichierSource.loc[dfFichierSource.annee.apply(lambda x : float(x)<1950 if not (pd.isnull(x)) else True)]
dfFichierSource.loc[dfFichierSource.annee.apply(lambda x : float(x)<1950 if not (pd.isnull(x)) else True), 'annee']=None

In [388]:
dfFichierSource['trafic']=dfFichierSource['MJA TV (an'].apply(lambda x : x.replace(' ','').split('(')[0] if not pd.isnull(x) else None)
dfFichierSource['annee']=dfFichierSource['MJA TV (an'].apply(lambda x : x.replace(' ','').split('(')[1][:-1] if not pd.isnull(x) and '(' in x else None)
dfFichierSource['pcpl']=dfFichierSource['%PL'].apply(lambda x : float(x)*100 if not pd.isnull(x) else None)
dfFichierSource['nomRoute']=dfFichierSource['RD'].apply(lambda x : x.replace(' ','').replace('RD','D'))

In [396]:
dfFichierSource

In [407]:
dfConcatFixe.loc[dfConcatFixe.nomsource=='CD_046']

,trafic,pcpl,nomRoute,annee,geometry,nomsource,fichie_src
0,11723,NaN,D620,2014,POINT (574957.400 6375488.700),CD_046,geolocalisation_Cerema.shp
1,12260,4.46,D620,2014,POINT (576039.000 6372752.100),CD_046,geolocalisation_Cerema.shp
2,11610,NaN,D8,2011,POINT (573839.900 6375058.600),CD_046,geolocalisation_Cerema.shp
3,10750,NaN,D8,2009,POINT (574449.100 6374950.700),CD_046,geolocalisation_Cerema.shp
4,10348,3.12,D8,2009,POINT (574880.600 6373961.200),CD_046,geolocalisation_Cerema.shp
5,9020,3.97,D8,2015,POINT (575191.900 6374028.300),CD_046,geolocalisation_Cerema.shp
6,13589,NaN,D811,2012,POINT (574969.800 6375553.700),CD_046,geolocalisation_Cerema.shp
7,11225,5.96,D811,2018,POINT (574119.961 6376756.430),CD_046,geolocalisation_Cerema.shp
8,9417,6.28,D811,2018,POINT (571469.775 6379355.856),CD_046,geolocalisation_Cerema.shp
9,9442,NaN,D820,2018,POINT (579784.052 6423589.409),CD_046,geolocalisation_Cerema.shp


In [362]:
dfNettoyees

,dept,nom_fichier_trafic,valide,type,type_geom,nom_attr_trafic,nom_attr_pcpl,nom_attr_route,nom_attr_annee,projection,regles_gestion
0,002,geolocalisation_pt_comptage_Cerema.shp,oui,sig,point,TMJA,%PL,ROUTE,ANNÉE,Epsg:2154,non
1,008,BZ_Compil_trafic_2018 du 08.SHP,oui,sig,point,TMJA_18,PCT_PL_18,ROUTE,NaN,Epsg:2154,non
5,025,2019_12_12_Points_Comptage_CD25.shp,oui,sig,point,TMJA,%_PL,AXE,NaN,Epsg:2154,"Trafic, pcpl"
7,031,CD31-Haute-Garonne.shp,oui,sig,point,tmja_tra,pl_tra,route,annee_tra,Epsg:4326,route
11,039,RO_GP_COMPT_PONCT.shp,oui,sig,point,TV,TAUX_PL,AXE,ANNEE,Epsg:2154,non
25,070,Trafic 2010 - 2018 CD 70.shp,oui,sig,point,TA_TV18,POURCENT_P,ROUTE,Tous,Epsg:2154,non
33,089,2020-02-26__Trafic_Bruit(2018)_du_89.shp,oui,sig,point,"TMJA,N,9,0","%_PL,N,3,2","NOM_RUE,C,","DATE,D",Epsg:2154,non
